#### Copyright 2024 Google LLC.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

# https://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Setup

Follow [this guide](https://developers.google.com/zero-touch/guides/auth) to set up your service account.

Download a service account key from the [Google Cloud Console](https://console.cloud.google.com) in JSON format.

Upload this file using the 'Files' tab in the side menu of this colab, and rename the file to 'sa.json'.

In [1]:
#@title Set up the environment and authenticate with the service
import json
from apiclient import discovery
import httplib2
from oauth2client.service_account import ServiceAccountCredentials
from google.protobuf import field_mask_pb2

# A single auth scope is used for the zero-touch enrollment customer API.
SCOPES = ['https://www.googleapis.com/auth/androidworkprovisioning']
SERVICE_ACCOUNT_KEY_FILE = 'sa.json'

def get_credential():
  """Creates a Credential object with the correct OAuth2 authorization.

  Uses the service account key stored in SERVICE_ACCOUNT_KEY_FILE.

  Returns:
    Credentials, the user's credential.
  """
  credential = ServiceAccountCredentials.from_json_keyfile_name(
    SERVICE_ACCOUNT_KEY_FILE, SCOPES)

  if not credential or credential.invalid:
    print('Unable to authenticate using service account key.')
  return credential


def get_service():
  """Creates a service endpoint for the zero-touch enrollment API.

  Builds and returns an authorized API client service for v1 of the API. Use
  the service endpoint to call the API methods.

  Returns:
    A service Resource object with methods for interacting with the service.
  """
  http_auth = get_credential().authorize(httplib2.Http())
  return discovery.build('androiddeviceprovisioning', 'v1', http=http_auth)

# Create a zero-touch enrollment API service endpoint.
service = get_service()

PARTNER_ID = '595346643' #@param {type:"string"}

# partners.customers

In [ ]:
#@title [partners.customers.list](https://developers.google.com/zero-touch/reference/reseller/rest/v1/partners.customers/list)

service.partners().customers().list(partnerId=PARTNER_ID).execute()

# partners.devices

In [ ]:
#@title [partners.devices.claim](https://developers.google.com/zero-touch/reference/reseller/rest/v1/partners.devices/claim)
#@markdown This will claim the device using the IMEI and Customer ID

## CLAIM DEVICE
def claim(imei,customer_id):
  device_identifier = {
    'imei':imei
    }

  request_body = {
    'deviceIdentifier':device_identifier,
    'customerId':customer_id,
    'sectionType':'SECTION_TYPE_ZERO_TOUCH'
    }

  # Claim the device
  response = service.partners().devices().claim(partnerId=PARTNER_ID,body=request_body).execute()
  print(response)

#@markdown sample `device_imei` : `123456789012347`
device_imei = '123456789012347' #@param {type:"string"}
customer_id = '1520371398' #@param {type:"string"}
claim(device_imei,customer_id)

In [ ]:
#@title [partners.devices.unclaim](https://developers.google.com/zero-touch/reference/reseller/rest/v1/partners.devices/unclaim)
#@markdown This will unclaim the device using the IMEI
## UNCLAIM DEVICE
def unclaim(imei):
  # Wrap the hardware ID and manufacturer values in a DeviceIdentifier.
  # Then wrap the DeviceIdentifier in a DeviceReference.
  device_identifier = {
    'imei':imei
    }

  request_body = {
      'deviceIdentifier': device_identifier,
      'sectionType':'SECTION_TYPE_ZERO_TOUCH'
      }

  # Unclaim the device
  response = service.partners().devices().unclaim(partnerId=PARTNER_ID, body=request_body).execute()
  print(response)

#@markdown sample `device_imei` : `123456789012347`
device_imei = '123456789012347' #@param {type:"string"}
unclaim(device_imei)

In [ ]:
#@title [partners.devices.claimAsync](https://developers.google.com/zero-touch/reference/reseller/rest/v1/partners.devices/claimAsync)
#@markdown This will claim multiple devices using the IMEI and Customer ID

def claim(imeis,customer_id):
  partner_claims = []
  for imei in imeis:
    device_identifier = {
      'imei':imei
      }

    partner_claim = {
      'deviceIdentifier':device_identifier,
      'customerId':customer_id,
      'sectionType':'SECTION_TYPE_ZERO_TOUCH'
      }

    partner_claims.append(partner_claim)

  request_body = {
      'claims': partner_claims
  }

  # Claim the device
  response = service.partners().devices().claimAsync(partnerId=PARTNER_ID,body=request_body).execute()
  print(response)

device1_imei = '123456789012347' #@param {type:"string"}
device2_imei = '123456789012348' #@param {type:"string"}
customer_id = '1520371398' #@param {type:"string"}
claim([device1_imei, device2_imei],customer_id)

In [ ]:
#@title [partners.devices.unclaimAsync](https://developers.google.com/zero-touch/reference/reseller/rest/v1/partners.devices/unclaimAsync)
#@markdown This will unclaim the device using the IMEI
## UNCLAIM DEVICE
def unclaim_async(imeis):
  partner_unclaims = []
  for imei in imeis:
    # Wrap the hardware ID and manufacturer values in a DeviceIdentifier.
    # Then wrap the DeviceIdentifier in a DeviceReference.
    device_identifier = {
      'imei':imei
      }

    partner_unclaim = {
        'deviceIdentifier': device_identifier,
        'sectionType':'SECTION_TYPE_ZERO_TOUCH'
        }

    partner_unclaims.append(partner_unclaim)

  request_body = {
      'unclaims': partner_unclaims
  }

  # Unclaim the device
  response = service.partners().devices().unclaimAsync(partnerId=PARTNER_ID, body=request_body).execute()
  print(response)

#@markdown sample `device_imei` : `123456789012347`
device1_imei = '123456789012347' #@param {type:"string"}
device2_imei = '123456789012348' #@param {type:"string"}
unclaim_async([device1_imei, device2_imei])

In [ ]:
#@title [partners.devices.get](https://developers.google.com/zero-touch/reference/reseller/rest/v1/partners.devices/get)
#@markdown This gets the device details

def get_device(device_id):
  print('get device')
  device_name='partners/'+PARTNER_ID+'/devices/'+device_id
  response = service.partners().devices().get(name=device_name).execute()
  print (response)

#@markdown sample `device_id` : `4037820647906988512`
device_id = '' #@param {type:"string"}
get_device(device_id)

In [ ]:
#@title [partners.devices.findByIdentifier](https://developers.google.com/zero-touch/reference/reseller/rest/v1/partners.devices/findByIdentifier)
#@markdown This finds a device using the device's identifiers

def find_by_identifier(device_imei):
  print('find by identifier')
  device = {
      'imei': device_imei
  }
  request_body = {
      'limit': 100,
      'deviceIdentifier': device
  }
  response = service.partners().devices().findByIdentifier(partnerId=PARTNER_ID, body=request_body).execute()
  print (response)

#@markdown sample `device_imei` : `123456789012347`
device_imei = '123456789012347' #@param {type:"string"}
find_by_identifier(device_imei)

In [ ]:
#@title [partners.devices.findByOwner](https://developers.google.com/zero-touch/reference/reseller/rest/v1/partners.devices/findByOwner)
#@markdown This finds a device using the customer ID

def find_by_owner(customer_id):
  print('find by owner')
  request_body = {
      'limit': 100,
      'sectionType': 'SECTION_TYPE_ZERO_TOUCH',
      'customerId': customer_id
  }
  response = service.partners().devices().findByOwner(partnerId=PARTNER_ID, body=request_body).execute()
  print (response)

customer_id = '1520371398' #@param {type:"string"}
find_by_owner(customer_id)

# operations

In [ ]:
#@title [operations.get](https://developers.google.com/zero-touch/reference/reseller/rest/v1/operations/get)
#@markdown This gets the status of a long-running operation

def get_operation(operation_name):
  print('get operation')
  response = service.operations().get(name=operation_name).execute()
  print (response)

#@markdown sample `operation_name` : `operations/apibatchoperation/5231233147924387049`
operation_name = 'operations/apibatchoperation/5231233147924387049' #@param {type:"string"}
get_operation(operation_name)